In [2]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.16.125:43943
distributed.scheduler - INFO -   dashboard at:                     :8787


In [13]:
## path for mdules
import sys
sys.path.insert(0,"/home/jovyan/xscale")
import xscale


sys.path.insert(0,"/home/jovyan/eNATL60-plots-paper/common-lib/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time
import pandas as pd

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

In [4]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [5]:
zmap_grid = fs.get_mapper('pangeo-meom/eNATL60-I/eNATL60-mesh-mask')
ds_grid = xr.open_zarr(zmap_grid)


In [6]:
navlat= ds_grid['nav_lat']
navlon= ds_grid['nav_lon']
e1u=ds_grid.e1u[0]
e1v=ds_grid.e1v[0]
e2u=ds_grid.e2u[0]
e2v=ds_grid.e2v[0]
ff=ds_grid.ff[0]
fmask=ds_grid.fmask[0,0]

In [10]:
def boxcar_factor_test(array2D,icrs=3,jcrs=3):
    """Test whether the shape of array2D is suited to coarsening with icrs,jcrs
    """
    jpj, jpi = array2D.shape
    if jpj%jcrs==0 and jpi%icrs==0:
       return True
    else:
       return False

def boxcar_reshape(array2D,icrs=3,jcrs=3):
    """Return a 3D array where values in boxes added in extra dimensions 
    """
    if not(boxcar_factor_test(array2D,icrs=icrs,jcrs=jcrs)):
       print("shape and coarsening factors are not compatible")
       return
    jpj, jpi = array2D.shape
    # target shape is shape = (jcrs, icrs, jpj/jcrs, jpi/icrs)
    t = np.reshape(array2D,(jpj,-1,icrs)) 		# (jpj, jpi/icrs, icrs)
    tt = t.swapaxes(0,2)				# (icrs,jpi/icrs, jpi)
    ttt = np.reshape(tt,(icrs,jpi//icrs,-1,jcrs)) 	# (icrs,jpi/icrs,jpj/jcrs, jcrs)
    tttt = ttt.swapaxes(1,3)  				# (icrs,jcrs,jpj/jcrs, jpi/icrs)
    ttttt = tttt.swapaxes(0,1) 				# (jcrs,icrs,jpj/jcrs, jpi/icrs)
    return ttttt

def boxcar_ravel(array2D,icrs=3,jcrs=3):
    """Return a 3D array where values in boxes are broadcasted along the third axis.
        output shape is (icrs*jcrs,jpj_crs,jpi_csr)
    """
    if not(boxcar_factor_test(array2D,icrs=icrs,jcrs=jcrs)):
       print("shape and coarsening factors are not compatible")
       return
    reshaped = boxcar_reshape(array2D,icrs=icrs,jcrs=jcrs)
    dum,dum,jpj,jpi = reshaped.shape
    raveled = reshaped.reshape((icrs*jcrs,jpj,jpi))
    return raveled

def coarse_var(var):
    jpj,jpi = var.shape
    x_offset=0
    y_offset=0
    crs_factor=60
    jcrs, icrs = crs_factor, crs_factor
    jsize = jpj - (jpj - y_offset) % jcrs
    isize = jpi - (jpi - x_offset) % icrs     
    cut_array = lambda array2D:array2D[...,y_offset:jsize,x_offset:isize]
    rvl = lambda a:boxcar_ravel(a,icrs=crs_factor,jcrs=crs_factor)  
    var_cut=cut_array(var.squeeze())
    varcrs=rvl(var_cut.values)
    varcrsm=varcrs.mean(axis=0)
    return varcrsm

In [ ]:
    t = np.reshape(array2D,(jpj,-1,icrs)) 		# (jpj, jpi/icrs, icrs)
    tt = t.swapaxes(0,2)				# (icrs,jpi/icrs, jpi)
    ttt = np.reshape(tt,(icrs,jpi//icrs,-1,jcrs)) 	# (icrs,jpi/icrs,jpj/jcrs, jcrs)
    tttt = ttt.swapaxes(1,3)  				# (icrs,jcrs,jpj/jcrs, jpi/icrs)
    ttttt = tttt.swapaxes(0,1) 				# (jcrs,icrs,jpj/jcrs, jpi/icrs)


In [7]:
    var=navlat
    jpj,jpi = var.shape
    x_offset=0
    y_offset=0
    crs_factor=60
    jcrs, icrs = crs_factor, crs_factor
    jsize = jpj - (jpj - y_offset) % jcrs
    isize = jpi - (jpi - x_offset) % icrs     
    cut_array = lambda array2D:array2D[...,y_offset:jsize,x_offset:isize]
    navlat_cut=cut_array(navlat)

In [8]:
navlat_cut_array=navlat_cut.values

In [12]:
jpj, jpi = navlat_cut_array.shape
print(jpj,jpi)
t = np.reshape(navlat_cut_array,(jpj,-1,icrs))
print(t.shape)
tt = t.swapaxes(0,2)				# (icrs,jpi/icrs, jpi)
print(tt.shape)
ttt = np.reshape(tt,(icrs,jpi//icrs,-1,jcrs)) 	# (icrs,jpi/icrs,jpj/jcrs, jcrs)
print(ttt.shape)
tttt = ttt.swapaxes(1,3)  				# (icrs,jcrs,jpj/jcrs, jpi/icrs)
print(tttt.shape)
ttttt = tttt.swapaxes(0,1) 				# (jcrs,icrs,jpj/jcrs, jpi/icrs)
print(ttttt.shape)


4680 8340
(4680, 139, 60)
(60, 139, 4680)
(60, 139, 78, 60)
(60, 60, 78, 139)
(60, 60, 78, 139)


In [9]:
jpj, jpi = navlat_cut.shape

In [14]:
navlat_cut

<xarray.DataArray 'nav_lat' (y: 4680, x: 8340)>
dask.array<getitem, shape=(4680, 8340), dtype=float32, chunksize=(4680, 8340), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x

In [18]:
[np.arange(139),np.arange(60)]

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,

In [16]:
import pandas as pd
numbers=[np.arange(139),np.arange(60)]
colors=['xcrs', 'icrs']
ind=pd.MultiIndex.from_product([numbers, colors],
                           names=['number', 'color'])

TypeError: unhashable type: 'numpy.ndarray'

In [10]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """


    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [14]:
test_reshape=xr_reshape(navlat_cut,'x',['xcrs','icrs'],[np.arange(139),np.arange(60)])

In [21]:
test_reshape

<xarray.DataArray 'nav_lat' (y: 4680, xcrs: 139, icrs: 60)>
dask.array<reshape, shape=(4680, 139, 60), dtype=float32, chunksize=(4680, 139, 60), chunktype=numpy.ndarray>
Coordinates:
  * xcrs     (xcrs) int64 0 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137 138
  * icrs     (icrs) int64 0 1 2 3 4 5 6 7 8 9 ... 50 51 52 53 54 55 56 57 58 59
Dimensions without coordinates: y

In [15]:
ttest_reshape=xr_reshape(test_reshape,'y',['ycrs','jcrs'],[np.arange(78),np.arange(60)])

In [23]:
ttest_reshape

<xarray.DataArray 'nav_lat' (ycrs: 78, jcrs: 60, xcrs: 139, icrs: 60)>
dask.array<transpose, shape=(78, 60, 139, 60), dtype=float32, chunksize=(78, 60, 139, 60), chunktype=numpy.ndarray>
Coordinates:
  * xcrs     (xcrs) int64 0 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137 138
  * icrs     (icrs) int64 0 1 2 3 4 5 6 7 8 9 ... 50 51 52 53 54 55 56 57 58 59
  * ycrs     (ycrs) int64 0 1 2 3 4 5 6 7 8 9 ... 68 69 70 71 72 73 74 75 76 77
  * jcrs     (jcrs) int64 0 1 2 3 4 5 6 7 8 9 ... 50 51 52 53 54 55 56 57 58 59

In [24]:
test_stack=ttest_reshape.stack(z=('jcrs','icrs'))

In [25]:
test_stack

<xarray.DataArray 'nav_lat' (ycrs: 78, xcrs: 139, z: 3600)>
dask.array<reshape, shape=(78, 139, 3600), dtype=float32, chunksize=(78, 139, 3600), chunktype=numpy.ndarray>
Coordinates:
  * xcrs     (xcrs) int64 0 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137 138
  * ycrs     (ycrs) int64 0 1 2 3 4 5 6 7 8 9 ... 68 69 70 71 72 73 74 75 76 77
  * z        (z) MultiIndex
  - jcrs     (z) int64 0 0 0 0 0 0 0 0 0 0 0 ... 59 59 59 59 59 59 59 59 59 59
  - icrs     (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 50 51 52 53 54 55 56 57 58 59

distributed.scheduler - INFO - Remove worker tcp://10.32.50.2:36661
distributed.core - INFO - Removing comms to tcp://10.32.50.2:36661
distributed.scheduler - INFO - Remove worker tcp://10.32.50.3:36023
distributed.core - INFO - Removing comms to tcp://10.32.50.3:36023


In [16]:
test_stack=ttest_reshape.stack(ijcrs=('jcrs','icrs'))

In [17]:
test_stack

<xarray.DataArray 'nav_lat' (ycrs: 78, xcrs: 139, ijcrs: 3600)>
dask.array<reshape, shape=(78, 139, 3600), dtype=float32, chunksize=(78, 139, 3600), chunktype=numpy.ndarray>
Coordinates:
  * xcrs     (xcrs) int64 0 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137 138
  * ycrs     (ycrs) int64 0 1 2 3 4 5 6 7 8 9 ... 68 69 70 71 72 73 74 75 76 77
  * ijcrs    (ijcrs) MultiIndex
  - jcrs     (ijcrs) int64 0 0 0 0 0 0 0 0 0 0 ... 59 59 59 59 59 59 59 59 59 59
  - icrs     (ijcrs) int64 0 1 2 3 4 5 6 7 8 9 ... 50 51 52 53 54 55 56 57 58 59

In [31]:
def boxcar_factor_test(array2D,icrs=3,jcrs=3):
    """Test whether the shape of array2D is suited to coarsening with icrs,jcrs
    """
    jpj, jpi = array2D.shape
    if jpj%jcrs==0 and jpi%icrs==0:
       return True
    else:
       return False

def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """
    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)
    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()
    A1.coords[dim] = ind
    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)
    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)
    for d in newdims[::-1]:
        dims.insert(i, d)
    for d in newdims:
        _ = dims.pop(-1)
    return A1.transpose(*dims)

def boxcar_reshape(array2D,icrs,jcrs,isize,jsize):
    """Return a 3D array where values in boxes added in extra dimensions 
    """
    if not(boxcar_factor_test(array2D,icrs=icrs,jcrs=jcrs)):
       print("shape and coarsening factors are not compatible")
       return
    jpj, jpi = array2D.shape
    # target shape is shape = (jcrs, icrs, jpj/jcrs, jpi/icrs)
    t=xr_reshape(array2D,'x',['xcrs','icrs'],[np.arange(isize),np.arange(icrs)])
    tt=xr_reshape(t,'y',['ycrs','jcrs'],[np.arange(jsize),np.arange(jcrs)]) 
    ttt=tt.stack(ijcrs=('jcrs','icrs'))
    return ttt

def coarse_var(var):
    jpj,jpi = var.shape
    x_offset=0
    y_offset=0
    crs_factor=60
    jcrs, icrs = crs_factor, crs_factor
    jsize = jpj - (jpj - y_offset) % jcrs
    isize = jpi - (jpi - x_offset) % icrs
    inb=(jpi - x_offset) // icrs
    jnb=(jpj - y_offset) // jcrs
    cut_array = lambda array2D:array2D[...,y_offset:jsize,x_offset:isize]
    var_cut=cut_array(var.squeeze())
    var_rechunk=var_cut.chunk({'y':jnb,'x':inb})
    varcrs=boxcar_reshape(var_rechunk,crs_factor,crs_factor,inb,jnb)
    varcrsm=varcrs.mean(dim='ijcrs')
    return varcrsm

In [32]:
%%time
ff_coarse=coarse_var(ff)

CPU times: user 518 ms, sys: 28 ms, total: 546 ms
Wall time: 532 ms


distributed.scheduler - INFO - Register tcp://10.32.24.2:35215
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.2:35215
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.24.3:39219
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.3:39219
distributed.core - INFO - Starting established connection


In [33]:
def boxcar_factor_test(array2D,icrs=3,jcrs=3):
    """Test whether the shape of array2D is suited to coarsening with icrs,jcrs
    """
    jpj, jpi = array2D.shape
    if jpj%jcrs==0 and jpi%icrs==0:
       return True
    else:
       return False

def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """
    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)
    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()
    A1.coords[dim] = ind
    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)
    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)
    for d in newdims[::-1]:
        dims.insert(i, d)
    for d in newdims:
        _ = dims.pop(-1)
    return A1.transpose(*dims)

def boxcar_reshape(array2D,icrs,jcrs,isize,jsize):
    """Return a 3D array where values in boxes added in extra dimensions 
    """
    if not(boxcar_factor_test(array2D,icrs=icrs,jcrs=jcrs)):
       print("shape and coarsening factors are not compatible")
       return
    jpj, jpi = array2D.shape
    # target shape is shape = (jcrs, icrs, jpj/jcrs, jpi/icrs)
    t=xr_reshape(array2D,'x',['xcrs','icrs'],[np.arange(isize),np.arange(icrs)])
    tt=xr_reshape(t,'y',['ycrs','jcrs'],[np.arange(jsize),np.arange(jcrs)]) 
    ttt=tt.stack(ijcrs=('jcrs','icrs'))
    return ttt

def coarse_var_chunk(var):
    jpj,jpi = var.shape
    x_offset=0
    y_offset=0
    crs_factor=60
    jcrs, icrs = crs_factor, crs_factor
    jsize = jpj - (jpj - y_offset) % jcrs
    isize = jpi - (jpi - x_offset) % icrs
    inb=(jpi - x_offset) // icrs
    jnb=(jpj - y_offset) // jcrs
    cut_array = lambda array2D:array2D[...,y_offset:jsize,x_offset:isize]
    var_cut=cut_array(var.squeeze())
    var_rechunk=var_cut.chunk({'y':jsize,'x':isize})
    varcrs=boxcar_reshape(var_rechunk,crs_factor,crs_factor,inb,jnb)
    varcrsm=varcrs.mean(dim='ijcrs')
    return varcrsm

In [34]:
%%time
ff_coarse=coarse_var_chunk(ff)

CPU times: user 31.7 ms, sys: 5.41 ms, total: 37.1 ms
Wall time: 30.8 ms
